In [1]:
pip install pubchempy

In [2]:
pip install rdkit

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pubchempy as pcp
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors

def obtener_masa_y_elementos(smiles):
    mol = Chem.MolFromSmiles(smiles)
    masa = rdMolDescriptors.CalcExactMolWt(mol)
    elementos = {}
    for atom in mol.GetAtoms():
        simbolo = atom.GetSymbol()
        elementos[simbolo] = elementos.get(simbolo, 0) + 1
    return masa, elementos

def main():
    print("Cálculo de masa en gramos de un compuesto A presente en un compuesto B")

    # ---------- Compuesto A ----------
    A = input("Introduce la fórmula del compuesto A: ")
    compuesto_A = pcp.get_compounds(A, "formula")
    if not compuesto_A:
        print(f"No se encontró el compuesto A: {A}")
        return

    smiles_A = compuesto_A[0].isomeric_smiles
    masa_A, elementos_A = obtener_masa_y_elementos(smiles_A)
    print(f"Masa molecular exacta de {A}: {masa_A:.4f} g/mol")

    # ---------- Compuesto B ----------
    B = input("Introduce la fórmula del compuesto B: ")
    compuesto_B = pcp.get_compounds(B, "formula")
    if not compuesto_B:
        print(f"No se encontró el compuesto B: {B}")
        return

    smiles_B = compuesto_B[0].isomeric_smiles
    masa_B, elementos_B = obtener_masa_y_elementos(smiles_B)
    print(f"Masa molecular exacta de {B}: {masa_B:.4f} g/mol")

    # ---------- Verificar que todos los elementos de A están en B ----------
    for elemento in elementos_A:
        if elemento not in elementos_B:
            print(f"⚠️ El elemento {elemento} no se encuentra en {B}. No se puede continuar.")
            return

    # ---------- Calcular relación estequiométrica ----------
    min_ratio = float('inf')
    for elemento, cantidad_A in elementos_A.items():
        cantidad_B = elementos_B[elemento]
        ratio = cantidad_B / cantidad_A
        min_ratio = min(min_ratio, ratio)

    # ---------- Cálculo de masa ----------
    try:
        masa_total_B = float(input(f"Introduce la masa de {B} en gramos: "))
    except ValueError:
        print("Entrada inválida para masa.")
        return

    moles_B = masa_total_B / masa_B
    moles_A = moles_B * min_ratio
    masa_total_A = moles_A * masa_A

    print(f"\nMasa de {A} presente en {masa_total_B} g de {B}: {masa_total_A:.2f} g")

# Ejecutar
main()

Cálculo de masa en gramos de un compuesto A presente en un compuesto B


Introduce la fórmula del compuesto A:  K


Masa molecular exacta de K: 38.9632 g/mol


Introduce la fórmula del compuesto B:  KCl


Masa molecular exacta de KCl: 73.9326 g/mol


Introduce la masa de KCl en gramos:  25



Masa de K presente en 25.0 g de KCl: 13.18 g
